In [1]:
print('****************REPLICAÇÃO DE PRODUTOS****************')

from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import pandas as pd
import numpy as np
import os
from time import time
from datetime import datetime

caminho_base = os.path.join(*os.getcwd().split('\\')[:3])

****************REPLICAÇÃO DE PRODUTOS****************


In [2]:
def produtos_disponiveis():
    chave_id = '7C2E97AA-1BC0-4C1F-9340-EC565A4520E4'
    url = 'http://ws.livekpl.onclick.com.br/AbacosWSplataforma.asmx'
    envelope_soup = f'''
            <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
              <soap:Body>
                <ProdutosDisponiveis xmlns="http://www.kplsolucoes.com.br/ABACOSWebService">
                  <ChaveIdentificacao>{chave_id}</ChaveIdentificacao>
                </ProdutosDisponiveis>
              </soap:Body>
            </soap:Envelope>'''
    
    headers = {'Content-Type': 'text/xml; charset=utf-8'}
    
    r = requests.post(url, data = envelope_soup, headers = headers)
    return r.text

def confirmar_produto(protocolo):
    chave_id = '7C2E97AA-1BC0-4C1F-9340-EC565A4520E4'
    url = 'http://ws.livekpl.onclick.com.br/AbacosWSplataforma.asmx'
    envelope_soup = f'''
            <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
              <soap:Body>
                <ConfirmarRecebimentoProduto xmlns="http://www.kplsolucoes.com.br/ABACOSWebService">
                  <ProtocoloProduto>{protocolo}</ProtocoloProduto>
                </ConfirmarRecebimentoProduto>
              </soap:Body>
            </soap:Envelope>'''
    
    headers = {'Content-Type': 'text/xml; charset=utf-8'}
    
    r = requests.post(url, data = envelope_soup, headers = headers)
    
def monitorar_integracao():
    chave_id = '7C2E97AA-1BC0-4C1F-9340-EC565A4520E4'
    url = 'http://ws.livekpl.onclick.com.br/AbacosWSplataforma.asmx'
    envelope_soup = f'''
        <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
          <soap:Body>
            <MonitoriaIntegracao xmlns="http://www.kplsolucoes.com.br/ABACOSWebService">
              <ChaveIdentificacao>{chave_id}</ChaveIdentificacao>
            </MonitoriaIntegracao>
          </soap:Body>
        </soap:Envelope>'''
    
    headers = {'Content-Type': 'text/xml; charset=utf-8'}
    
    r = requests.post(url, data = envelope_soup, headers = headers)
    return r


#Criando função para coletar os preços com base numa lista de SKUs
def preco_online(lista_skus):
    lista_produtos = []
    for produto in lista_skus:
        lista_produtos.append(f'<string>{produto}</string>')
    
    chave_id = '7C2E97AA-1BC0-4C1F-9340-EC565A4520E4'
    url = 'http://ws.livekpl.onclick.com.br/AbacosWSplataforma.asmx'
    envelope_soup = f"""
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <PrecoOnLine xmlns="http://www.kplsolucoes.com.br/ABACOSWebService">
      <ChaveIdentificacao>{chave_id}</ChaveIdentificacao>
      <ListaDeCodigosProdutos>
      {lista_produtos}
      </ListaDeCodigosProdutos>
    </PrecoOnLine>
  </soap:Body>
</soap:Envelope>"""

    headers = {'Content-Type': 'text/xml; charset=utf-8'}

    r = requests.post(url, data = envelope_soup, headers = headers, timeout=500)
    return BeautifulSoup(r.text, 'xml')

In [3]:
while len(BeautifulSoup(produtos_disponiveis(), 'xml').find_all('DadosProdutos')) > 0:
    
    # print('Produtos na fila: ' + BeautifulSoup(monitorar_integracao().text, 'xml').find_all('DadosMonitoriaIntegracao')[0].find('QtdeRegistrosPendentes').text)
    
    produtos = pd.read_parquet(r'{0}\Vitrine Direta Dropbox\Vitrine Direta\17. Tech\Dashboard\Base de dados Dash\Dados atualizados\produtos kpl.gzip'.format(caminho_base).replace('C:', 'C:\\').replace('\\\\', '\\'))
    
    resposta = produtos_disponiveis()
    resposta = BeautifulSoup(resposta, 'xml')
    lista_dict = []
    
    for i in range(0, len(resposta.find_all('DadosProdutos'))):
        linha = {}
        produto = resposta.find_all('DadosProdutos')[i]
        linha['ProtocoloProduto'] = produto.find_all('ProtocoloProduto')[0].text
        linha['Acao'] = produto.find_all('Acao')[0].text
        linha['CodigoProdutoAbacos'] = produto.find_all('CodigoProdutoAbacos')[0].text
        linha['CodigoProduto'] = produto.find_all('CodigoProduto')[0].text
        linha['CodigoProdutoPai'] = produto.find_all('CodigoProdutoPai')[0].text
        linha['CodigoBarras'] = produto.find_all('CodigoBarras')[0].text
        
        linha['TipoProduto'] = produto.find_all('TipoProduto')[0].text
        linha['NomeProduto'] = produto.find_all('NomeProduto')[0].text
        linha['NomeProdutoReduzido'] = produto.find_all('NomeProdutoReduzido')[0].text
        linha['Descricao'] = produto.find_all('Descricao')[0].text
        linha['CodigoMarca'] = produto.find_all('CodigoMarca')[0].text
        linha['CodigoFabricante'] = produto.find_all('CodigoFabricante')[0].text
        linha['CodigoClasse'] = produto.find_all('CodigoClasse')[0].text
        linha['CodigoExternoClasse'] = produto.find_all('CodigoExternoClasse')[0].text
        linha['CodigoFamilia'] = produto.find_all('CodigoFamilia')[0].text
        linha['CodigoExternoFamilia'] = produto.find_all('CodigoExternoFamilia')[0].text
        linha['CodigoGrupo'] = produto.find_all('CodigoGrupo')[0].text
        linha['CodigoSubGrupo'] = produto.find_all('CodigoSubGrupo')[0].text
        linha['Peso'] = produto.find_all('Peso')[0].text
        linha['Largura'] = produto.find_all('Largura')[0].text
        linha['Comprimento'] = produto.find_all('Comprimento')[0].text
        linha['Profundidade'] = produto.find_all('Profundidade')[0].text
        linha['Espessura'] = produto.find_all('Espessura')[0].text
        linha['Altura'] = produto.find_all('Altura')[0].text
        linha['QtdePorEmbalagem'] = produto.find_all('QtdePorEmbalagem')[0].text
        linha['QtdeMinimaEstoque'] = produto.find_all('QtdeMinimaEstoque')[0].text
        linha['QtdeMaximaEstoque'] = produto.find_all('QtdeMaximaEstoque')[0].text
        linha['UnidadeMedidaNome'] = produto.find_all('UnidadeMedidaNome')[0].text
        linha['UnidadeMedidaAbrev'] = produto.find_all('UnidadeMedidaAbrev')[0].text
        linha['UnidadeMedidaQuilo'] = produto.find_all('UnidadeMedidaQuilo')[0].text
        linha['ProdutoTemFilhos'] = produto.find_all('ProdutoTemFilhos')[0].text
        linha['PrazoEntregaDias'] = produto.find_all('PrazoEntregaDias')[0].text
        linha['OrigemMercadoria'] = produto.find_all('OrigemMercadoria')[0].text
        linha['CFOPNaoContribuinte'] = produto.find_all('CFOPNaoContribuinte')[0].text
        linha['CFOPContribuinte'] = produto.find_all('CFOPContribuinte')[0].text
        linha['UnidadeTributavel'] = produto.find_all('UnidadeTributavel')[0].text
        linha['CustoDoProduto'] = produto.find_all('CustoDoProduto')[0].text
        linha['CodigoExternoFornecedor'] = produto.find_all('CodigoExternoFornecedor')[0].text
        linha['DescontoMaxProduto'] = produto.find_all('DescontoMaxProduto')[0].text
        linha['PreVenda'] = produto.find_all('PreVenda')[0].text
        linha['DataInicio'] = produto.find_all('PreVenda_DataInicio')[0].text
        linha['PreVenda_DataFinal'] = produto.find_all('PreVenda_DataFinal')[0].text
        linha['CodigoProdutoParceiro'] = produto.find_all('CodigoProdutoParceiro')[0].text
        linha['CodigoProdutoPaiParceiro'] = produto.find_all('CodigoProdutoPaiParceiro')[0].text
        
        linha['Cor'] = produto.find_all('DadosDescritorPreDefinido')[0].find('Descricao').text
        try:
            linha['Tamanho'] = produto.find_all('DadosDescritorPreDefinido')[1].find('Descricao').text
        except:
            linha['Tamanho'] = np.nan
        
        if produto.find_all('DadosCaracteristicasComplementares')[0].find('TipoGrupoNome').text.strip() == 'DESCRIÇÕES PARA O SITE':
            linha['Descrição'] = produto.find_all('DadosCaracteristicasComplementares')[0].find('Texto').text
        
        try:
            linha['Garantia'] = produto.find('DiasGarantia').text.strip()
            linha['Indicação'] = produto.find_all('DadosCaracteristicasComplementares')[2].find('Texto').text
            linha['Medidas'] = produto.find_all('DadosCaracteristicasComplementares')[3].find('Texto').text
            linha['DESCRIÇÕES PARA O SITE'] = produto.find_all('DadosCaracteristicasComplementares')[4].find('Texto').text
            linha['Lojas'] = produto.find_all('DadosCaracteristicasComplementares')[5].find('Texto').text
        except:
            linha['Garantia'] = np.nan
            linha['Indicação'] = np.nan
            linha['Medidas'] = np.nan
            linha['DESCRIÇÕES PARA O SITE'] = np.nan
            linha['Lojas'] = np.nan

        #Dados de produtos
        linha['Classe'] = produto.find('DescricaoClasse').text
        linha['Marca'] = produto.find('DescricaoMarca').text.strip()
        linha['Família'] = produto.find('DescricaoFamilia').text.strip()
        linha['Grupo'] = produto.find('DescricaoGrupo').text
        linha['Subgrupo'] = produto.find('DescricaoSubgrupo').text.strip()
        linha['Categoria fiscal'] = produto.find('CodigoCategoriaFiscal').text.strip()
        linha['CodigoClassificacaoFiscal'] = produto.find('CodigoClassificacaoFiscal').text
        linha['Classificação fiscal'] = produto.find('ClassificacaoFiscal').text.strip()
        
        
        lista_dict.append(linha)
    
    produtos_novo = pd.DataFrame(lista_dict)
    
    produtos = produtos[~produtos.CodigoProduto.isin(produtos_novo.CodigoProduto)].reset_index(drop = True)
    produtos = pd.concat([produtos, produtos_novo], ignore_index = True)

    produtos = produtos.drop_duplicates().reset_index(drop = True)
    produtos['Coleta'] = datetime.today()
    
    
    #Corrigindo formato das colunas antes de salvar em Parquet
    l1 = ['CodigoProdutoAbacos', 'CodigoBarras', 'CodigoMarca', 'CodigoClasse', 'CodigoFamilia', 'CodigoGrupo', 'CodigoSubGrupo', 'ProdutoTemFilhos', 'Categoria fiscal', 'UnidadeMedidaQuilo', 'OrigemMercadoria', 'CFOPNaoContribuinte', 'CFOPContribuinte', 'UnidadeTributavel', 'DESCRIÇÕES PARA O SITE']
    l2 = ['Peso', 'Largura', 'Altura', 'Profundidade', 'Espessura', 'Comprimento', 'CustoDoProduto', 'DescontoMaxProduto', 'PrazoEntregaDias']
    l3 = ['QtdePorEmbalagem', 'QtdeMaximaEstoque', 'QtdeMinimaEstoque']
    lista = [(l1, 'str'), (l2, 'float'), (l3, 'int')]

    for colunas, tipo in lista:
        for coluna in colunas:
            produtos[coluna] = produtos[coluna].astype(tipo)

    
    #Coletando o preço da tabela de preços padrão
    a = time()
    print('\nColetando os preços dos SKUS...')
    resposta = preco_online(list(produtos['CodigoProduto']))
    lista_skus, lista_precos = [], []
    for produto in resposta.find_all('DadosPrecoResultado'):
        lista_skus.append(produto.find('CodigoProduto').text)
        lista_precos.append(float(produto.find('PrecoTabela').text))
    df_preco = pd.DataFrame()
    df_preco['SKU'] = lista_skus
    df_preco['Preço'] = lista_precos
    produtos['Preço'] = pd.merge(left = produtos, right = df_preco, left_on = 'CodigoProduto', right_on = 'SKU', how = 'left')['Preço']
    b = time()
    print(f'Coleta de preços realizada em {str(round(b - a, 2))} seg')
    
    
    #Salvando arquivo
    produtos.to_parquet(r'{0}\Vitrine Direta Dropbox\Vitrine Direta\17. Tech\Dashboard\Base de dados Dash\Dados atualizados\produtos kpl.gzip'.format(caminho_base).replace('C:', 'C:\\'), index = False, compression = 'gzip')
    
    
    print('\nConfirmando produtos')
    for i in tqdm(range(0, len(produtos_novo))):
        confirmar_produto(produtos_novo.iloc[i]['ProtocoloProduto'])
        pass
    
from plyer import notification
notification.notify(title='Programa Executado', message='Fim', app_name='Produtos Disponíveis', timeout=3)


Coletando os preços dos SKUS...
Coleta de preços realizada em 13.95 seg

Confirmando produtos


100%|██████████████████████████████████████████████████████████████████████████████████| 86/86 [00:05<00:00, 16.88it/s]



Coletando os preços dos SKUS...
Coleta de preços realizada em 18.85 seg

Confirmando produtos


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.26it/s]



Coletando os preços dos SKUS...
Coleta de preços realizada em 15.75 seg

Confirmando produtos


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 16.24it/s]



Coletando os preços dos SKUS...
Coleta de preços realizada em 17.01 seg

Confirmando produtos


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14.97it/s]
